In [4]:
from numba import cuda, njit, vectorize, guvectorize, prange, float64
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy.signal as signal

# load some data

In [2]:
data = loadmat('./data/sim_single_cyst.mat')
print(data.keys())

fs = np.squeeze(data['fs']) # sampling frequency
fs = np.float64(fs)
c = np.squeeze(data['c']) # speed of sound
# when c is uint16 it raise error, thus it should be casted into proper type
c = np.float64(c)

pitch = np.squeeze(data['pitch']) # pitch
angles = np.squeeze(data['angles']) # transmit angles

rf = np.squeeze(data['rf']) # radio-frequency (rf) data acquired by probe elements
rf = np.transpose(rf,(2,1,0))
# decimation - the fs=1e8 is not necessary high
dec = 4
rf = signal.decimate(rf, dec)
fs = fs/dec

print(f'sampling frequency [MHz]: {fs*1e-6}')


dict_keys(['__header__', '__version__', '__globals__', 'angles', 'c', 'fc', 'fs', 'pitch', 'rf', 'rfrc'])
sampling frequency [MHz]: 25.0


***
# FIR filter
***
<font size="4">
The finite impulse response (FIR) filter is given by equation \eqref{fir}

\begin{equation}
    y[n] = \sum_{k=0}^{N-1} h[k]x[n-k] 
    \label{fir} \tag{1}
\end{equation}

where *x* is an input signal, *h* is a filter impulse response and *y* is an output signal.
Filtration is realised by convolving the signal with impulse response of the filter.

  
    
***    
</font>

# FIR filter implementation - 1D (naive approach)

### define some functions

In [80]:
def present_signal(sig, fs, color='b', title=''):
    '''
    The function display input signal and its spectrum.
    '''
    f, p = signal.periodogram(sig,fs=fs)
    
    fig,ax = plt.subplots(nrows=1, ncols=2)
    fig.suptitle(title)
    ax[0].plot(sig,color=color)
    ax[0].set_title('signal')
    ax[0].set_xlabel('samples')
    ax[0].set_ylabel('amplitude [a.u.]')
    
    ax[1].semilogy(f*1e-6, p/np.max(psig),color=color)
    ax[1].set_title('power spectrum')
    ax[1].set_xlabel('frequency [MHz]')
    ax[1].set_ylabel('amplitude [a.u.]')
    
    plt.tight_layout()
    
    
def convolve(sig, h):
    '''
    The function convolve signals (sig) with filter impulse response (h).
    It convolve along the last dimension of sig array. 
    '''
    
    s = sig.shape
    sfshape = s[0:-1] + (s[-1] - h.shape[-1],)
    sigflt = np.zeros(sfshape)
    
    for i in range(sigflt.shape[-1]):
        sigflt[..., i] = np.sum(sig[...,i:i+h.size]*h[::-1])
        
    return sigflt

convolve_jit = njit()(convolve)


# experiment
@guvectorize([(float64[:], float64[:], float64[:])], '(),()->()')
def convolve_guv(sig, h, sigflt):
    '''
    The function convolve two signals.
    '''
#     sigflt = np.zeros(sig.size - h.size)
    for i in range(sigflt.size):
        sigflt[i] = np.sum(sig[i:i+h.size]*h[::-1])
        

    
    

In [82]:
# get some signal from rf array
sig = rf[0,0,:]

# create filter impulse response (filter coefficient array)
ntaps = 256
h = np.concatenate((np.linspace(0, 1, ntaps//2),
                    np.linspace(1, 0, ntaps//2)))

# ordinary convolve
%time sigflt = convolve(sig, h)

# convolve compiled by jit (the time gain will be visible after second run)
%time sigflt = convolve_jit(sig, h)

# sigflt2 = np.zeros(sig.shape)
# print(sigflt2.shape)
# convolve_guv(sig, h, sigflt2)







# # do some graphs
# fig,ax = plt.subplots(nrows=1, ncols=1)
# ax.plot(h)
# ax.set_title('filter impulse response')
# present_signal(sig, fs, 'blue', 'input signal')
# present_signal(sigflt, fs, 'red', 'filtered signal')


CPU times: user 15.5 ms, sys: 0 ns, total: 15.5 ms
Wall time: 15.3 ms
CPU times: user 1.09 ms, sys: 0 ns, total: 1.09 ms
Wall time: 1.1 ms


In [78]:
10//3

a = (1,2,3)
b = a[0:-1]+(10,)
print(b)

(1, 2, 10)
